In [26]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


from scipy.stats import chi2_contingency

In [27]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

In [28]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

Before (891, 12) (418, 11) (891, 12) (418, 11)


('After', (891, 10), (418, 9), (891, 10), (418, 9))

In [29]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,C,3
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,7.9250,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,53.1000,S,3
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,8.0500,S,1


In [30]:
contingency_table = pd.crosstab(train_df['Sex'], train_df['Title'])
chi2, p, dof, ex = chi2_contingency(contingency_table)
print(p)

4.448184462993702e-189


Because 'Sex' and 'Title' column is numerical but they are not continous variables. So it is impossible to check multicollinearity by calculating VIF(variance inflation factor). We could use Chi-square test to check the relationship.
4.448184462993702e-189 is very small value which is close to 0. It is smaller than 0.05 so we could reject H0 : 'Sex', 'Title' variables are independent. 
So we found out that 'Sex', 'Title' are not independent and are strongly associated.

In [31]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

In [32]:
guess_ages = np.zeros((2,3))
guess_ages

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22,1,0,7.2500,S,1
1,1,1,1,38,1,0,71.2833,C,3
2,1,3,1,26,0,0,7.9250,S,2
3,1,1,1,35,1,0,53.1000,S,3
4,0,3,0,35,0,0,8.0500,S,1


In [33]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 7)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 11.429]",0.573529
1,"(11.429, 22.857]",0.409756
2,"(22.857, 34.286]",0.314448
3,"(34.286, 45.714]",0.426829
4,"(45.714, 57.143]",0.411765
5,"(57.143, 68.571]",0.346154
6,"(68.571, 80.0]",0.142857


In [34]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 7)

age_stats = (
    train_df.groupby('AgeBand')
    .agg(
        Survived_mean=('Survived', 'mean'),
        Count=('Survived', 'size')
    )
    .reset_index()
    .sort_values(by='AgeBand')
)

print(age_stats)

            AgeBand  Survived_mean  Count
0   (-0.08, 11.429]       0.573529     68
1  (11.429, 22.857]       0.409756    205
2  (22.857, 34.286]       0.314448    353
3  (34.286, 45.714]       0.426829    164
4  (45.714, 57.143]       0.411765     68
5  (57.143, 68.571]       0.346154     26
6    (68.571, 80.0]       0.142857      7


In [35]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 23), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 23) & (dataset['Age'] <= 34), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 34) & (dataset['Age'] <= 46), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 46) & (dataset['Age'] <= 57), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 57) & (dataset['Age'] <= 69), 'Age'] = 5
    dataset.loc[ dataset['Age'] > 69, 'Age'] = 6

train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,1,1,0,7.2500,S,1
1,1,1,1,3,1,0,71.2833,C,3
2,1,3,1,2,0,0,7.9250,S,2
3,1,1,1,3,1,0,53.1000,S,3
4,0,3,0,3,0,0,8.0500,S,1


In [36]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1,7.2500,0,1,0,3
1,1,1,1,3,71.2833,1,3,0,3
2,1,3,1,2,7.9250,0,2,1,6
3,1,1,1,3,53.1000,0,3,0,3
4,0,3,0,3,8.0500,0,1,1,9


In [37]:
fare0 = (train_df['Fare'] == 0).sum()
print(fare0)

15


In [38]:
train_df.loc[train_df['Fare'] == 0, 'Fare'] = np.nan
test_df.loc[test_df['Fare'] == 0, 'Fare'] = np.nan

for dataset in [train_df, test_df]:
    dataset['Fare'] = dataset.groupby(['Pclass', 'Embarked'])['Fare'].transform(lambda x: x.fillna(x.median()))

In [39]:
fare0 = (train_df['Fare'] == 0).sum()
print(fare0)

0


In [40]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 5)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(4.010999999999999, 7.896]",0.206731
1,"(7.896, 11.242]",0.253333
2,"(11.242, 23.0]",0.416667
3,"(23.0, 41.579]",0.426136
4,"(41.579, 512.329]",0.627119


In [41]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 5)

fare_stats = (
    train_df.groupby('FareBand')
    .agg(
        Survived_mean=('Survived', 'mean'),
        Total=('Survived', 'size')
    )
    .reset_index()
    .sort_values(by='FareBand')
)

print(fare_stats)


                     FareBand  Survived_mean  Total
0  (4.010999999999999, 7.896]       0.206731    208
1             (7.896, 11.242]       0.253333    150
2              (11.242, 23.0]       0.416667    180
3              (23.0, 41.579]       0.426136    176
4           (41.579, 512.329]       0.627119    177


In [42]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 7)

fare_stats = (
    train_df.groupby('FareBand')
    .agg(
        Survived_mean=('Survived', 'mean'),
        Total=('Survived', 'size')
    )
    .reset_index()
    .sort_values(by='FareBand')
)

print(fare_stats)


                     FareBand  Survived_mean  Total
0  (4.010999999999999, 7.775]       0.219858    141
1               (7.775, 8.05]       0.203008    133
2                (8.05, 13.0]       0.335664    143
3              (13.0, 19.967]       0.397849     93
4            (19.967, 28.918]       0.436508    126
5            (28.918, 56.929]       0.445312    128
6           (56.929, 512.329]       0.685039    127


In [43]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.775, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.775)  & (dataset['Fare'] <= 8.050),   'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 8.050)  & (dataset['Fare'] <= 13.000),  'Fare'] = 2
    dataset.loc[(dataset['Fare'] > 13.000) & (dataset['Fare'] <= 19.967),  'Fare'] = 3
    dataset.loc[(dataset['Fare'] > 19.967) & (dataset['Fare'] <= 28.918),  'Fare'] = 4
    dataset.loc[(dataset['Fare'] > 28.918) & (dataset['Fare'] <= 56.929),  'Fare'] = 5
    dataset.loc[ dataset['Fare'] > 56.929,                                'Fare'] = 6
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]   
    
train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1,0,0,1,0,3
1,1,1,1,3,6,1,3,0,3
2,1,3,1,2,1,0,2,1,6
3,1,1,1,3,5,0,3,0,3
4,0,3,0,3,1,0,1,1,9


In [44]:
print(train_df['Survived'].value_counts())

print(train_df['Survived'].value_counts(normalize=True) * 100)

0    549
1    342
Name: Survived, dtype: int64
0    61.616162
1    38.383838
Name: Survived, dtype: float64


In [45]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [23]:
'''
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

X_train, Y_train = smote.fit_resample(X_train, Y_train)

print(Y_train.value_counts())
'''

0    549
1    549
Name: Survived, dtype: int64


In [46]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)


coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)

# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)

# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)

# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)

# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)

# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

C:\Users\hanji\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\hanji\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [47]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,88.89
8,Decision Tree,88.89
1,KNN,84.62
0,Support Vector Machines,82.94
2,Logistic Regression,80.92
6,Stochastic Gradient Decent,80.47
7,Linear SVC,80.25
5,Perceptron,80.02
4,Naive Bayes,77.33


In [48]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
# submission.to_csv('../output/submission.csv', index=False)